In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/flt/Sound")

# Music genre classification notebook
## 1. Import libraries

In [4]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#keras
import keras


Using TensorFlow backend.


## 2.Extracting features from spectrogram
We will extract

    -Mel frequency cepstral coefficients(MFCC)
    -spectral centroid
    -zero cross rate
    -chroma frequencies
    -spectral roll-off

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00081.au,0.380260,0.248262,2116.942959,1956.611056,4196.107960,0.127272,-26.929785,107.334008,-46.809993,40.932634,-21.463776,24.816275,-18.945571,15.253459,-15.050104,12.262823,-15.234050,14.336612,-13.821769,7.562789,-6.181372,0.330165,-6.829571,0.965922,-7.570825,2.918987,blues
1,blues.00022.au,0.306451,0.113475,1156.070496,1497.668176,2170.053545,0.058613,-233.860772,136.170239,3.289490,48.931193,8.699106,13.954752,5.359449,0.381334,-13.490265,16.871769,5.145258,-2.250578,3.959198,5.322555,0.812028,-1.107202,-4.556555,-2.436490,3.316913,-0.608485,blues
2,blues.00031.au,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802549,110.843071,18.620984,30.922996,3.291330,22.587447,-20.418303,12.120995,-26.347956,-4.356270,-26.406674,-13.037723,-12.652228,-1.821905,-7.260097,-6.660252,-14.682694,-11.719264,-11.025216,-13.387260,blues
3,blues.00012.au,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208080,132.799175,-15.438986,60.986727,0.725807,12.437828,1.183836,-1.540779,-17.888154,8.358496,-2.452068,-0.613248,0.384877,2.605128,-5.188924,-9.527455,-9.244394,-2.848274,-1.418707,-5.932607,blues
4,blues.00056.au,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434568,102.829023,-12.517677,56.993472,2.271347,26.591402,-4.274803,17.338809,-7.985593,15.235427,-6.934599,7.457218,-10.470444,-2.360483,-6.783624,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues


In [6]:
print(data.shape)
data = data.drop('filename',axis=1)

(1000, 28)


### Encoding Labels

In [0]:
genere_list = data.iloc[:,-1]
le = LabelEncoder()
y = le.fit_transform(genere_list)

### Sacling feature columns

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,:-1],dtype=float))

### Dividing data into train test set

In [15]:
x_train, x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
print(len(x_train))
print(len(x_test))
print(x_train[10])

800
200
[ 1.68664855  0.59821279  0.24312126 -0.02014807  0.20778073  0.10449413
  0.77686193 -0.35801561 -0.85847302  1.0131924  -0.65829091  1.12557918
 -0.29908318  1.54522776 -0.14884932  0.62337006 -0.16233764  1.37096774
 -0.57745557  1.23457507 -0.49711913  1.52179455 -0.2205741   1.46221494
  0.72101749  0.98982254]


### Model building

In [31]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Dense(512, activation='relu',input_shape = (x_train.shape[1],)))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train,
          y_train,
          epochs=30,
          batch_size=150)

Epoch 1/30
800/800 [==============================] - 1s 890us/step - loss: 2.1253 - acc: 0.3000
Epoch 2/30
800/800 [==============================] - 0s 84us/step - loss: 1.7046 - acc: 0.4063
Epoch 3/30
800/800 [==============================] - 0s 85us/step - loss: 1.3884 - acc: 0.5175
Epoch 4/30
800/800 [==============================] - 0s 97us/step - loss: 1.2154 - acc: 0.5763
Epoch 5/30
800/800 [==============================] - 0s 95us/step - loss: 1.0652 - acc: 0.6338
Epoch 6/30
800/800 [==============================] - 0s 88us/step - loss: 0.9758 - acc: 0.6612
Epoch 7/30
800/800 [==============================] - 0s 86us/step - loss: 0.8961 - acc: 0.6975
Epoch 8/30
800/800 [==============================] - 0s 83us/step - loss: 0.8212 - acc: 0.7263
Epoch 9/30
800/800 [==============================] - 0s 92us/step - loss: 0.7974 - acc: 0.7150
Epoch 10/30
800/800 [==============================] - 0s 89us/step - loss: 0.7214 - acc: 0.7488
Epoch 11/30
800/800 [=================

In [32]:
test_loss, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

200/200 [==============================] - 0s 1ms/step
0.635


The accuracy is less than train accuracy, it indicates overfitting

### validation approach
lets apart 200 observation from train set to act as vali8dation set

In [0]:
x_val = x_train[:200]
x_partial_train = x_train[200:]

y_val = y_train[:200]
y_partial_train = y_train[200:]

In [36]:
model.fit(x_partial_train,
          y_partial_train,
          epochs=30,
          batch_size = 512,
          validation_data = (x_val,y_val))
result = model.evaluate(x_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 78us/step - loss: 0.1208 - acc: 0.9717 - val_loss: 0.1408 - val_acc: 0.9650
Epoch 2/30
600/600 [==============================] - 0s 69us/step - loss: 0.1439 - acc: 0.9600 - val_loss: 0.1049 - val_acc: 0.9800
Epoch 3/30
600/600 [==============================] - 0s 71us/step - loss: 0.1030 - acc: 0.9800 - val_loss: 0.1309 - val_acc: 0.9650
Epoch 4/30
600/600 [==============================] - 0s 65us/step - loss: 0.1203 - acc: 0.9717 - val_loss: 0.1149 - val_acc: 0.9750
Epoch 5/30
600/600 [==============================] - 0s 77us/step - loss: 0.0989 - acc: 0.9850 - val_loss: 0.1147 - val_acc: 0.9700
Epoch 6/30
600/600 [==============================] - 0s 80us/step - loss: 0.1012 - acc: 0.9850 - val_loss: 0.1116 - val_acc: 0.9700
Epoch 7/30
600/600 [==============================] - 0s 75us/step - loss: 0.0838 - acc: 0.9867 - val_loss: 0.1183 - val_acc: 0.9750
Epoch 8/30
600/600 [===

In [37]:
result

[1.472375783920288, 0.64]

# 4. Prediction on test dataset

In [43]:
prediction = model.predict(x_test)
print(prediction[0].shape)
print(np.argmax(prediction[0]))

(10,)
5
